<a href="https://colab.research.google.com/github/IsraelAbebe/Amharic-NLP/blob/master/Amharic-Predictive-text/Amharic-word-predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
from torch import nn

import numpy as np


In [0]:
text = ["sometimes when you feel buried you are actually just planted",
        "be still and know that i am God",
        "god is most glorified when we are most satisfied in him"]

In [21]:
chars = set(' '.join(text))
print(chars)

{'y', 'c', 'o', 'i', 'w', 'd', 'k', 'h', 'm', 'G', 'r', 'n', 's', 'e', 'b', 'u', 'g', ' ', 'j', 'l', 'a', 'p', 'f', 't'}


In [22]:
int2char = dict(enumerate(chars))
print(int2char)

{0: 'y', 1: 'c', 2: 'o', 3: 'i', 4: 'w', 5: 'd', 6: 'k', 7: 'h', 8: 'm', 9: 'G', 10: 'r', 11: 'n', 12: 's', 13: 'e', 14: 'b', 15: 'u', 16: 'g', 17: ' ', 18: 'j', 19: 'l', 20: 'a', 21: 'p', 22: 'f', 23: 't'}


In [23]:
char2int = {int2char[k]:k for k in int2char.keys()}
print(char2int)

{'y': 0, 'c': 1, 'o': 2, 'i': 3, 'w': 4, 'd': 5, 'k': 6, 'h': 7, 'm': 8, 'G': 9, 'r': 10, 'n': 11, 's': 12, 'e': 13, 'b': 14, 'u': 15, 'g': 16, ' ': 17, 'j': 18, 'l': 19, 'a': 20, 'p': 21, 'f': 22, 't': 23}


In [24]:
maxlen = len(max(text,key=len))
maxlen

60

In [125]:
for i in range(len(text)):
    while len(text[i]) < maxlen:
        text[i] += ' '

text

['sometimes when you feel buried you are actually just planted',
 'be still and know that i am God                             ',
 'god is most glorified when we are most satisfied in him     ']

In [126]:
input_sequence = []
target_sequence = []

for i in range(len(text)):
    input_sequence.append(text[i][:-1])
    target_sequence.append(text[i][1:])
    
    print(input_sequence[i],target_sequence[i])

sometimes when you feel buried you are actually just plante ometimes when you feel buried you are actually just planted
be still and know that i am God                             e still and know that i am God                             
god is most glorified when we are most satisfied in him     od is most glorified when we are most satisfied in him     


One hot mapping

In [127]:
for i in range(len(text)):
    input_sequence[i] = [char2int[character] for character in input_sequence[i]]
    target_sequence[i] = [char2int[character] for character in target_sequence[i]]

    print(input_sequence[i],target_sequence[i])

[12, 2, 8, 13, 23, 3, 8, 13, 12, 17, 4, 7, 13, 11, 17, 0, 2, 15, 17, 22, 13, 13, 19, 17, 14, 15, 10, 3, 13, 5, 17, 0, 2, 15, 17, 20, 10, 13, 17, 20, 1, 23, 15, 20, 19, 19, 0, 17, 18, 15, 12, 23, 17, 21, 19, 20, 11, 23, 13] [2, 8, 13, 23, 3, 8, 13, 12, 17, 4, 7, 13, 11, 17, 0, 2, 15, 17, 22, 13, 13, 19, 17, 14, 15, 10, 3, 13, 5, 17, 0, 2, 15, 17, 20, 10, 13, 17, 20, 1, 23, 15, 20, 19, 19, 0, 17, 18, 15, 12, 23, 17, 21, 19, 20, 11, 23, 13, 5]
[14, 13, 17, 12, 23, 3, 19, 19, 17, 20, 11, 5, 17, 6, 11, 2, 4, 17, 23, 7, 20, 23, 17, 3, 17, 20, 8, 17, 9, 2, 5, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17] [13, 17, 12, 23, 3, 19, 19, 17, 20, 11, 5, 17, 6, 11, 2, 4, 17, 23, 7, 20, 23, 17, 3, 17, 20, 8, 17, 9, 2, 5, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17]
[16, 2, 5, 17, 3, 12, 17, 8, 2, 12, 23, 17, 16, 19, 2, 10, 3, 22, 3, 13, 5, 17, 4, 7, 13, 11, 17, 4, 13

In [0]:
dict_size = len(int2char)
sequence_len = maxlen - 1
batch_size = len(text)


In [0]:
def one_hot_encode(sequence,dict_size,sequence_len,batch_size):
    features = np.zeros((batch_size,sequence_len,dict_size),dtype=np.float32)

    for i in range(batch_size):
        for u in range(sequence_len):
            features[i,u,sequence[i][u]] = 1
    
    return features

In [130]:
dict_size

24

In [0]:
input_sequence = one_hot_encode(input_sequence,dict_size,sequence_len,batch_size)

In [0]:
# target_sequence = one_hot_encode(target_sequence,dict_size,sequence_len,batch_size)

In [0]:
input_seq = torch.from_numpy(input_sequence)
target_seq = torch.tensor(target_sequence).squeeze_()

In [134]:
is_cuda = torch.cuda.is_available()
is_cuda

True

In [135]:
device = torch.device('cuda') if is_cuda else torch.device('cpu')
device

device(type='cuda')

In [0]:
class Model(nn.Module):
    def __init__(self,input_size,output_size,hidden_dim,n_layers):
        super(Model,self).__init__()

        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.rnn = nn.RNN(input_size,hidden_dim,n_layers,batch_first=True)
        self.fc = nn.Linear(hidden_dim,output_size)
    
    def forward(self,x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        out,hidden = self.rnn(x,hidden)
        out = out.contiguous().view(-1,self.hidden_dim)
        out = self.fc(out)

        return out,hidden

    def init_hidden(self,batch_size):
        hidden = torch.zeros(self.n_layers,batch_size,self.hidden_dim)
        return hidden


In [143]:
model = Model(input_size=dict_size,output_size=dict_size,hidden_dim=12,n_layers=1)
model#.to(device)
model

Model(
  (rnn): RNN(24, 12, batch_first=True)
  (fc): Linear(in_features=12, out_features=24, bias=True)
)

In [0]:
n_epoches = 1000
lr = 0.01

criterion  = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=lr)

In [153]:
for epoch in range(1,n_epoches+1):
    optimizer.zero_grad()
    input_seq#.to(device)
    output,hidden = model(input_seq)    
    loss = criterion(output,target_seq.view(-1).long())
    loss.backward()
    optimizer.step()

    if epoch%10 == 0 :
        print('Epoch : {}/{}...'.format(epoch,n_epoches),end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch : 10/1000... Loss: 0.6468
Epoch : 20/1000... Loss: 0.5382
Epoch : 30/1000... Loss: 0.4443
Epoch : 40/1000... Loss: 0.3648
Epoch : 50/1000... Loss: 0.3057
Epoch : 60/1000... Loss: 0.2602
Epoch : 70/1000... Loss: 0.2187
Epoch : 80/1000... Loss: 0.1827
Epoch : 90/1000... Loss: 0.1833
Epoch : 100/1000... Loss: 0.2793
Epoch : 110/1000... Loss: 0.1852
Epoch : 120/1000... Loss: 0.1457
Epoch : 130/1000... Loss: 0.1221
Epoch : 140/1000... Loss: 0.1070
Epoch : 150/1000... Loss: 0.0950
Epoch : 160/1000... Loss: 0.0850
Epoch : 170/1000... Loss: 0.0762
Epoch : 180/1000... Loss: 0.0683
Epoch : 190/1000... Loss: 0.0616
Epoch : 200/1000... Loss: 0.2275
Epoch : 210/1000... Loss: 0.2158
Epoch : 220/1000... Loss: 0.1431
Epoch : 230/1000... Loss: 0.1007
Epoch : 240/1000... Loss: 0.0786
Epoch : 250/1000... Loss: 0.0672
Epoch : 260/1000... Loss: 0.0591
Epoch : 270/1000... Loss: 0.0530
Epoch : 280/1000... Loss: 0.0483
Epoch : 290/1000... Loss: 0.0445
Epoch : 300/1000... Loss: 0.0413
Epoch : 310/1000...

In [0]:
def predict(model,character):
    character = np.array([[char2int[c] for c in character]])
    character = one_hot_encode(character,dict_size,character.shape[1],1)
    character = torch.from_numpy(character)
    
    out, hidden = model(character)

    prob  = nn.functional.softmax(out[-1],dim=0).data
    char_ind = torch.max(prob,dim=0)[1].item()

    return int2char[char_ind], hidden

In [0]:
def sample(model,out_len,start='hey'):
    model.eval()
    start = start.lower()
    chars = [i for i in start]
    size = out_len-len(chars)

    for i in range(size):
        char,h = predict(model,chars)
        chars.append(char)

    return ''.join(chars)

In [157]:
sample(model, 15, 'what')

'whatied in mest'